In [1]:
#Kütüphaneleri sisteme yüklüyoruz.
import cv2 #görüntü işleme, video analizi kütüphanesi
import mediapipe as mp #makine öğrenimi kütüphanesi
import pyautogui  #fare ve klavyenizi kontrol etmeyi sağlayan kütüphane

In [2]:
cam=cv2.VideoCapture(0)  #Cv2 den video yakalamasını istiyoruz. 0 değerini verince kendi kameramızdan yakalama yapıyor.
face_mesh=mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)
screen_w, screen_h=pyautogui.size() #Size komuzu ekran çözünürlük bilgimizi bize geri dönüyor. bunu screen_w, screen_h olarak tanımlıyoruz
while True:
    _,frame=cam.read() #kamera okuma tanımlamalarımızı belli ediyoruz. 
    frame=cv2.flip(frame,1) #Kameranın ayna yansıma özelliğini, yani yatay olarak ters olmasını düzeltiyoruz.
    rgb_frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #Renkli görüntü tespiti etmek için BGR'ı RGB'ye çeviriyoruz. 
    output=face_mesh.process(rgb_frame) #Çıktımızı Rgb aldığımızı belirtiyoruz.
    landmarks_points=output.multi_face_landmarks #tanımlamamızı yüzün noktalarına eşitliyoruz. Yüzün tamamını görmesi lazım xyz değerler vermesi için.
    frame_h, frame_w,_ =frame.shape #Frame in (camera da okunana yerin boyutu) yükseklik değeri ve genişlik değerini alıyoruz.
    pyautogui.PAUSE = 0.00000001 #Pytautogui'de mouse hızını arttırmak için, mouse hareket bekleme süresini düşüğe çekiyoruz.
    if landmarks_points:
        landmarks = landmarks_points[0].landmark #landmarks_
        for id, landmark in enumerate(landmarks[474:478]): #Yüzdeki belirli yerlerin numarası vardır. sağ gözün 4 noktasını alıyoruz. 474 den 477 e kadar olan kısmı.478 dahil değil.
                                                           #Enumerate kullanarak 2 değere dönüştürüyoruz. 2 Değer ile sınıflandırıp çıktı alıyoruz.
            x=int(landmark.x * frame_w)    #X eksenini frame'in genişliği ile çarpıyoruz. Decimal sayı olarak çıktı alıyor. Decimal sayı virgülden sonra çok olduğu için integer'a çeviriyoruz.
            y=int(landmark.y * frame_h)    #Y eksenini frame'in yüksekliği ile çarpıyoruz. Decimal sayı olarak çıktı alıyor. Decimal sayı virgülden sonra çok olduğu için integer'a çeviriyoruz.
            cv2.circle(frame,(x,y), 3,(0,255,0))  #x ve y nin orta orta noktasını bulup, frame üzerinden yeşil renkte , yüzdeki noktaları belirler.
            if id==1:  
                screen_x = screen_w / frame_w*x  #Ekran genişliği ile frame'in genişliğini bölüp X ekseni ile çarpıyoruz.
                screen_y= screen_h/ frame_h * y  #Ekran yükseliği ile frame'in yüksekliğini bölüp Y ekseni ile çarpıyoruz.
                pyautogui.moveTo(screen_x,screen_y)  #mouse 'u screen_x ve screen_y koordinatlarına hareketlendirmeyi tanımlıyoruz.

        left = [landmarks[145],landmarks[159]]    #Sol gözün Göz kapağının alt kısmını (145) belirtiyoruz ve üst kısmını (159) belirtiyoruz.
        for landmark in left:   #landmark noktalarını sol gözde dolaştırıyoruz.
            x=int(landmark.x * frame_w)  #X eksenini frame'in genişliği ile çarpıyoruz. Decimal sayı olarak çıktı alıyor. Decimal sayı virgülden sonra çok olduğu için integer'a çeviriyoruz.
            y=int(landmark.y * frame_h)  #Y eksenini frame'in yüksekliği ile çarpıyoruz. Decimal sayı olarak çıktı alıyor. Decimal sayı virgülden sonra çok olduğu için integer'a çeviriyoruz.
            cv2.circle(frame,(x,y), 3,(0,255,255)) #x ve y nin orta orta noktasını bulup, frame üzerinden sarı renkte , yüzdeki noktaları belirler.
            
        if(left[0].y - left[1].y)< 0.005: #Left dizisinin ilk değeri yani göz kapağının altı ile 2. değeri göz kapağının üstünü y kordinatında çıkartıyoruz. 
                                          #0.005 den küçükse aşağıdaki işlemleri yaptırıyoruz.
            pyautogui.click()  #tıklama yaptırıyoruz
            pyautogui.sleep(1) #1 sn bekletiyoruz.

        mouth = [landmarks[17],landmarks[0]] #Alt dudak (17) ile üst dudağı(0) tanımlıyoruz.
        for landmark in mouth: #Yüz noktalarını ağızdan bulduruyoruz.
            x=int(landmark.x * frame_w)  #X eksenini frame'in genişliği ile çarpıyoruz. Decimal sayı olarak çıktı alıyor. Decimal sayı virgülden sonra çok olduğu için integer'a çeviriyoruz.
            y=int(landmark.y * frame_h)  #Y eksenini frame'in yüksekliği ile çarpıyoruz. Decimal sayı olarak çıktı alıyor. Decimal sayı virgülden sonra çok olduğu için integer'a çeviriyoruz.
            cv2.circle(frame,(x,y), 3,(255,0,0)) #alt dudak ve üst dudaktaki noktaları renklendiriyoruz.
                
        if(mouth[0].y - mouth[1].y) > 0.08 : # alt dudağın y eksenini , üst dudağın y ekseninden çıkarıyoruz.
            pyautogui.doubleClick()  #tıklama yaptırıyoruz
            pyautogui.sleep(1) #1 sn bekletiyoruz.


    cv2.imshow('Göz ile Mouse Kontrolü ',frame)  #çerçeve başlığımız

    if cv2.waitKey(10) & 0xFF==ord('q'):  # 0.10sn boyunca q tuşuna basılınca kapatma komutu veriyoruz.
        break #kır
cam.release() #video almayı kapatır.
cv2.destroyAllWindows() #açılan bütün pencereleri yok eder.
    